In [1]:
# dependencies
using Polynomials

# First Implementation 
Direction: 
- Evaluate the function at various points around the edges of the rectangle. 
- Count how many cycles the argument completes while traversing around the rectangle.

Spot rectangle, and how many zeros are inside.
By tracking the winding number, we can locate the number of zeros in the holomorphic function

Wind(arg) = 1/(2*pi*im) * integrate(1/(x+im*y))

In [2]:
# define data structure of rectangle
struct Rec
    btmlft::Any
    uplft::Any
    uprght::Any
    btmrght::Any
end


struct Step
    height::Float32
    width::Float32
    step::Float32
end

In [104]:
# calculate height and width
function parseInput(input)
    height = input.uplft[2] - input.btmlft[2]
    width = input.btmrght[1] - input.btmlft[1]
    n = 20000 # change n accordingly so argument analysis is comprehensive
    step = height/n
    info = Step(height, width, step)
end

parseInput (generic function with 1 method)

In [75]:
# evaluate the argument at various points starting at bottomleft and traversing one round
# store arguments in array

function argBox(f, input, info)
    # bottom left to top left
    i = input.btmlft[2]
    argArray = []
    while i < input.uplft[2] 
        arg = angle(f(input.btmlft[1]+im*i))  # this is where the input function comes in for evaluation. Check multipoint.ipynb
        push!(argArray,arg)
        i += info.step
    end
    
    # top left to top right
    j = input.uplft[1]
    while j < input.uprght[1]
        arg = angle(f(j+im*input.uplft[2]))
        push!(argArray,arg)
        j += info.step
    end
    
    # top right to bottom right
    i = input.uprght[2]
    while i > input.btmrght[2]
        arg = angle(f(input.uprght[1]+im*i))
        push!(argArray,arg)
        i -= info.step
    end
    
    # bottom right to bottom left
    j = input.btmrght[1]
    while j > input.btmlft[1]
        arg = angle(f(j+im*input.btmlft[2]))
        push!(argArray,arg)
        j -= info.step
    end
    
    return argArray
end


argBox (generic function with 1 method)

In [93]:
# spot jumps (~>= 2pi or 6)
# the output of argBox(input) is an array of the arguments
# use a for loop to check if any of the pairs match the criteria
# increase count and return the final count. This is the number of zeros in the locus


function countJump(f, input, info)
    arr = argBox(f, input, info)
    len = length(arr)
    count = 0
    add = []
    dec = []
    
    for i in 1:(len - 1)
        if arr[i] - arr[i+1] > 6 # slightly less than 2pi
            count += 1
            push!(add, (arr[i],arr[i+1]))
        elseif arr[i] - arr[i+1] < -6 # decrement if it goes clockwise (less than -6)
            count -= 1
            push!(dec, (arr[i],arr[i+1]))
        end
    end
    return count, add, dec
end

countJump (generic function with 1 method)

## Unit Tests
Test the algorithm properly. Polynomials are easy to put the zeros exactly where you want them. Try multiplying another function which has no zero in the region to make it more computationally expensive.

In [105]:
# first unit test with 4 zeros
t1 = fromroots([-1+2im,2+4im,-3im, 4+6im])
input = Rec((1,1), (1,10), (10,10), (10,1))
info = parseInput(input) # handles steps
countJump(t1, input, info) # == 2 #(1,4), (4,6)

(-1, Any[(3.14159, -3.14147)], Any[(-3.14128, 3.14157), (-3.14155, 3.1415)])

In [106]:
t2 = fromroots([1+2im,1+4im,3im, 4+6im]) * Polynomial([1, 2, 3], :x)
input2 = Rec((-10,0), (-10,10), (0,10), (0,0))
info2 = parseInput(input2)
countJump(t2, input2, info2) # == 1

(-2, Any[], Any[(-3.14144, 3.14154), (-3.1414, 3.14149)])